# Laboratorio 4. Análisis fundamental 
## GDP Price Index

Karla Pérez

 El laboratorio trata de probar cómo afecta la parte del análisis fundamental en este caso con la noticia de un indicador de desempeño del país,al tipo de cambio.
 


Paqueterias  que se deben instalar.

In [14]:
rm(list = ls())

# los 0s aceptados antes de expresas una cifra en notaci?n cient?fica
options("scipen"=100, "digits"=4)
mdir <- getwd()
suppressMessages(library(TTR)) 
# -- Establecer el sistema de medicion de la computadora
Sys.setlocale(category = "LC_ALL", locale = "")

###librerias
suppressMessages(library(plotly))
suppressMessages(library(xlsx))# 
suppressMessages(library(Quandl)) # Descarga de Precios
suppressMessages(library(PortfolioAnalytics)) ##Teoria modenar de portafolio
suppressMessages(library(ROI)) # optimización para ele portafolio 
suppressMessages(library(knitr))  
suppressMessages(library(kableExtra)) # Tablas en HTML
options(knitr.table.format = "html") 
Historicos<-(read.xlsx(file="C:/Users/perezka/OneDrive - HP/Karla/Karla/Trading/myst_lab4_KarlaPerez/History.xlsx", sheetName="History",
                                                            colIndex=1:4,startRow=1,endRow=46,header=TRUE))
Sys.setenv(tz="America/Monterrey", TZ="America/Monterrey")
options(tz="America/Monterrey", TZ="America/Monterrey")

# -- Cargar y/o instalar en automatico paquetes a utilizar -- #

pkg <- c("base","downloader","dplyr","fBasics","forecast","grid",
         "gridExtra","httr","jsonlite","lmtest","lubridate","moments",
         "matrixStats", "PerformanceAnalytics","plyr","quantmod",
         "reshape2","RCurl","RMySQL", "stats","scales","tseries",
         "TTR","TSA","XML","xts","zoo")

inst <- pkg %in% installed.packages()
if(length(pkg[!inst]) > 0) install.packages(pkg[!inst])
instpackages <- lapply(pkg, library, character.only=TRUE)

# -- Cargar archivos desde GitHub -- #

RawGitHub <- "https://raw.githubusercontent.com/IFFranciscoME/"
ROandaAPI <- paste(RawGitHub,"ROandaAPI/master/ROandaAPI.R",sep="")
downloader::source_url(ROandaAPI,prompt=FALSE,quiet=TRUE)

# -- Parametros para usar API-OANDA

# Tipo de cuenta practice/live
OA_At <- "practice"
# ID de cuenta
OA_Ai <- 1742531
# Token para llamadas a API
OA_Ak <- "ada4a61b0d5bc0e5939365e01450b614-4121f84f01ad78942c46fc3ac777baa6" 
# Hora a la que se considera "Fin del dia"
OA_Da <- 17
# Uso horario
OA_Ta <- "America/Mexico_City"
# Instrumento
OA_In <- "EUR_USD"
# Granularidad o periodicidad de los precios H4 = Cada 4 horas
OA_Pr <- "H4"
# Multiplicador de precios para convertir a PIPS
MultPIP_MT1 <- 10000


[1] "LC_COLLATE=English_United States.1252;LC_CTYPE=English_United States.1252;LC_MONETARY=English_United States.1252;LC_NUMERIC=C;LC_TIME=English_United States.1252"

Cargamos el excel de los datos históricos desde el 2007

In [15]:
Historicos<-(read.xlsx(file="C:/Users/perezka/OneDrive - HP/Karla/Karla/Trading/myst_lab4_KarlaPerez/History.xlsx", sheetName="History",
                                                            colIndex=1:4,startRow=1,endRow=46,header=TRUE))

In [16]:
Historicos<- as.data.frame(Historicos)
#limpiamos los valores Na de los historicos
Historicos[is.na(Historicos)] <- 0


Clasificación de Escenarios


In [17]:
Escenarios<- data.frame("Date" = Historicos$DateTime,
                        "Actual" = Historicos$Actual,
                        "Consensus"= Historicos$Consensus,
                        "Previous" = Historicos$Previous,
                        "Clasificacion" = NA )

#$Escenario.A<-which(Historicos$Actual>=Historicos$Consensus & Historicos$Consensus>=Historicos$Previous)
#clasificación 
for(j in 1:length(Historicos$DateTime)){#for rows
      if (Escenarios$Actual[j]>=Escenarios$Consensus[j]&Escenarios$Consensus[j]>=Escenarios$Previous[j]) {
         Escenarios$Clasificacion[j] <-"A"
      }#if A"
      else if (Escenarios$Actual[j]>=Escenarios$Consensus[j]&Escenarios$Consensus[j]<Escenarios$Previous[j]){
          Escenarios$Clasificacion[j] <-"B"
      }#if b
      else if (Escenarios$Actual[j]<Escenarios$Consensus[j]&Escenarios$Consensus[j]>=Escenarios$Previous[j]){
        Escenarios$Clasificacion[j] <-"C"
      }# c
      else {Escenarios$Clasificacion[j] <-"D"}

    
}# fin 

Escenarios



Date,Actual,Consensus,Previous,Clasificacion
2018-10-26 12:30:00,1.4,2.1,3.3,D
2018-09-27 12:30:00,3.3,3.0,3.2,B
2018-08-29 12:30:00,3.2,3.0,3.2,B
2018-07-27 12:30:00,3.2,2.3,2.2,A
2018-06-28 12:30:00,2.2,1.9,1.9,A
2018-05-30 12:30:00,1.9,2.0,2.0,C
2018-04-27 12:30:00,2.0,2.2,2.3,D
2018-03-28 12:30:00,2.3,2.3,2.3,A
2018-02-28 13:30:00,2.3,2.4,2.4,C
2018-01-26 13:30:00,2.4,2.3,2.1,A


En esta parte del código, ya convierte las fechas y hace la descarga del día de los precios del tipo de cambio, después  discrimina y guarda los 15 minutos antes y después del anuncio. 

In [20]:
data <- list()

Historicos$Date <- as.character(as.POSIXct(Historicos$Date,format = "%m/%d/%Y %H:%M")) # conviert todas las fechas a fromato Oanda

for ( i in 1:38){# forzamos a que sean los 3 años 

fecha_ejm <- Historicos$Date[[i]]
aux <- Historicos$Clasificacion[[i]]
  

#opcion convertir a a fecha

F2 <- as.Date(substr(fecha_ejm,1,10))

  
if(wday(F2) != 1) # if para saber en que dia se encuentra
  #wday 
  #si da 1, domingo
{
 
  Fecha1 <- F2
  Fecha2 <- F2+1
  Precios_Oanda <- HisPrices(AccountType = OA_At, Granularity = OA_Pr,
                             DayAlign = OA_Da, TimeAlign = OA_Ta, Token = OA_Ak,
                             Instrument = OA_In, 
                             Start = Fecha1, End = Fecha2, Count = NULL)
} else {
  
  Fecha1 <- F2-2
  Fecha2 <- F2+1
  Precios_Oanda <- HisPrices(AccountType = OA_At, Granularity = OA_Pr,
                             DayAlign = OA_Da, TimeAlign = OA_Ta, Token = OA_Ak,
                             Instrument = OA_In, 
                             Start = Fecha1, End = Fecha2, Count = NULL)
}

Precios_Oanda$TimeStamp <- as.character(as.POSIXct(Precios_Oanda$TimeStamp,format = "%m/%d/%Y %H:%M:%S"))

#convierte de nuevo la fecha del xlsx para poder encontrarla en el DataFrame

ind <- which(Precios_Oanda$TimeStamp == Historicos$Date[[i]])

data[[i]] <- list("Escenario" = Historicos$Clasificacion[i],"Precios" = Precios_Oanda[(ind-15):(ind+15),])
#print(paste0("Iteracion: ",i, " Fecha 1 es: ", Fecha1, " Fecha 2 es: ", Fecha2))
}

data

ERROR: Error in if (wday(F2) != 1) {: missing value where TRUE/FALSE needed


En esta parte trato de hacer la estimación de las medidas de desempeño por escenario

In [21]:
Calculos <-list("Rends"  = 0,
                "Desv_s" =  0,
                "Dif" = 0, 
                "max "=0)  

  Calculos$Rends<-data[,list(mean = mean(data$Precios$Close)), by = data$Escenario]
  
  Calculos$Desv_s<-data[,list(desv = stdev(data$Precios$Close)), by = data$Escenario]
  Calculos$Dif<-data[,list(dif= (data$Precios$Close[[1] -data$Precios$Close[[31]])), by = data$Escenario]
  Calculos$max<-date[,list(max = max(data$Precios$Close)- min(data$Precios$Close),by = data$Escenarios] 
                                
  


ERROR: Error in parse(text = x, srcfile = src): <text>:9:57: unexpected '-'
8:   Calculos$Desv_s<-data[,list(desv = stdev(data$Precios$Close)), by = data$Escenario]
9:   Calculos$Dif<-data[,list(dif= (data$Precios$Close[[1] -
                                                           ^


**El problema es que el api  no funciona acá en guatemala, no me deja descargar los precios de Oanda, o sólo algunos de los precios**

In [ ]:
rm(list = ls())

# los 0s aceptados antes de expresas una cifra en notaci?n cient?fica
options("scipen"=100, "digits"=4)
mdir <- getwd()
suppressMessages(library(TTR)) 
# -- Establecer el sistema de medicion de la computadora
Sys.setlocale(category = "LC_ALL", locale = "")

###librerias
suppressMessages(library(plotly))
suppressMessages(library(xlsx))# 
suppressMessages(library(Quandl)) # Descarga de Precios
suppressMessages(library(PortfolioAnalytics)) ##Teoria modenar de portafolio
suppressMessages(library(ROI)) # optimización para ele portafolio 
suppressMessages(library(knitr))  
suppressMessages(library(kableExtra)) # Tablas en HTML
options(knitr.table.format = "html") 

Historicos<-(read.csv(file="C:/Users/perezka/OneDrive - HP/Karla/Karla/Trading/myst_lab4_KarlaPerez/History.csv",header=TRUE))

Sys.setenv(tz="America/Monterrey", TZ="America/Monterrey")
options(tz="America/Monterrey", TZ="America/Monterrey")

# -- Cargar y/o instalar en automatico paquetes a utilizar -- #

pkg <- c("base","downloader","dplyr","fBasics","forecast","grid",
         "gridExtra","httr","jsonlite","lmtest","lubridate","moments",
         "matrixStats", "PerformanceAnalytics","plyr","quantmod",
         "reshape2","RCurl","RMySQL", "stats","scales","tseries",
         "TTR","TSA","XML","xts","zoo")

inst <- pkg %in% installed.packages()
if(length(pkg[!inst]) > 0) install.packages(pkg[!inst])
instpackages <- lapply(pkg, library, character.only=TRUE)

# -- Cargar archivos desde GitHub -- #

RawGitHub <- "https://raw.githubusercontent.com/IFFranciscoME/"
ROandaAPI <- paste(RawGitHub,"ROandaAPI/master/ROandaAPI.R",sep="")
downloader::source_url(ROandaAPI,prompt=FALSE,quiet=TRUE)

# -- Parametros para usar API-OANDA

# Tipo de cuenta practice/live
OA_At <- "practice"
# ID de cuenta
OA_Ai <- 1742531
# Token para llamadas a API
OA_Ak <- "ada4a61b0d5bc0e5939365e01450b614-4121f84f01ad78942c46fc3ac777baa6" 
# Hora a la que se considera "Fin del dia"
OA_Da <- 16
# Uso horario
OA_Ta <- "GMT"
# Instrumento
OA_In <- "USD_MXN"
# Granularidad o periodicidad de los precios H4 = Cada 4 horas
OA_Pr <- "M1"
# Multiplicador de precios para convertir a PIPS
MultPIP_MT1 <- 10000


Historicos$Date<-as.character(Historicos$Date)

Historicos[is.na(Historicos)] <- 0
Historicos<- data.frame("Date" = Historicos$Date,
                        "Actual" = Historicos$Actual,
                        "Consensus"= Historicos$Consensus,
                        "Previous" = Historicos$Previous,
                        "Clasificacion" = NA )

#$Escenario.A<-which(Historicos$Actual>=Historicos$Consensus & Historicos$Consensus>=Historicos$Previous)

for(j in 1:length(Historicos$Date)){#for rows
  if (Historicos$Actual[j]>=Historicos$Consensus[j]&Historicos$Consensus[j]>=Historicos$Previous[j]) {
    Historicos$Clasificacion[j] <-"A"
  }#if A"
  else if (Historicos$Actual[j]>=Historicos$Consensus[j]&Historicos$Consensus[j]<Historicos$Previous[j]){
    Historicos$Clasificacion[j] <-"B"
  }#if b
  else if (Historicos$Actual[j]<Historicos$Consensus[j]&Historicos$Consensus[j]>=Historicos$Previous[j]){
    Historicos$Clasificacion[j] <-"C"
  }# c
  else {Historicos$Clasificacion[j] <-"D"}
  
  
}# fin 



####Codigo para pedir los precios a OAnda fecha año-mes-dia
# tomar cada fecha y descargar las fechaas
data <- list()

  
Historicos$Date <- as.character(as.POSIXct(Historicos$Date,format = "%m/%d/%Y %H:%M")) # conviert todas las fechas a fromato Oanda

for ( i in 1:38){# forzamos a que sean los 3 años 

fecha_ejm <- Historicos$Date[[i]]
aux <- Historicos$Clasificacion[[i]]
  

#opcion convertir a a fecha

F2 <- as.Date(substr(fecha_ejm,1,10))

  
if(wday(F2) != 1) # if para saber en que dia se encuentra
  #wday 
  #si da 1, domingo
{
 
  Fecha1 <- F2
  Fecha2 <- F2+1
  Precios_Oanda <- HisPrices(AccountType = OA_At, Granularity = OA_Pr,
                             DayAlign = OA_Da, TimeAlign = OA_Ta, Token = OA_Ak,
                             Instrument = OA_In, 
                             Start = Fecha1, End = Fecha2, Count = NULL)
} else {
  
  Fecha1 <- F2-2
  Fecha2 <- F2+1
  Precios_Oanda <- HisPrices(AccountType = OA_At, Granularity = OA_Pr,
                             DayAlign = OA_Da, TimeAlign = OA_Ta, Token = OA_Ak,
                             Instrument = OA_In, 
                             Start = Fecha1, End = Fecha2, Count = NULL)
}

Precios_Oanda$TimeStamp <- as.character(as.POSIXct(Precios_Oanda$TimeStamp,format = "%m/%d/%Y %H:%M:%S"))

#convierte de nuevo la fecha del xlsx para poder encontrarla en el DataFrame

ind <- which(Precios_Oanda$TimeStamp == Historicos$Date[[i]])



data[[i]] <- list("Escenario" = Historicos$Clasificacion[i],"Precios" = Precios_Oanda[(ind-15):(ind+15),])
#print(paste0("Iteracion: ",i, " Fecha 1 es: ", Fecha1, " Fecha 2 es: ", Fecha2))

Calculos$Rends[[i]]<-data[,list(mean = mean(data[[i]]$Precios$Close)), by = data$Escenario]

Calculos$Desv_s[[i]]<-data[,list(desv = stdev(data[[i]]$Precios$Close)), by = data$Escenario]
Calculos$Dif[[i]]<-data[,list(dif= data$Precios$Close[[1]] - data$Precios$Close[[31]]), by = data$Escenario]
Calculos$max[[i]]<-date[,list(max = max(data$Precios$Close)-min(data$Precios$Close)),by = data$Escenarios] 


}
